###  Hugging Face Pipeline

In [1]:
# load model easily via pipeline method
from transformers import pipeline
checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
classifier = pipeline("sentiment-analysis", model=checkpoint)
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

### Behind the Pipeline

In [16]:
# let's recreate the above pipeline step by step
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# the tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# the model with a sequence classification head attached
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
# the inputs
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]

# toktenize the inputs
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print('Outputs from tokenizer:\n', inputs, "\n")

# run them through the model (with the sequence classification head)
outputs = model(**inputs)
print('Outputs from model:\n', outputs, "\n")

# obtain the predictions
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print("Model Predictions:\n", predictions,"\n")

# assign the labels to the predictions
print("Assing Labels to Predictions:")
id2label = model.config.id2label
for p in predictions:
    print({label:prob.item() for label, prob in zip(id2label.values(), p)})

Outputs from tokenizer:
 {'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])} 

Outputs from model:
 SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None) 

Model Predictions:
 tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>) 

Assing Labels to Predictions:

{'NEGATIVE': 0.040195148438215256, 'POSITIVE': 0.9598048329353333}
{'NEGATIVE': 0.9994558691978455, 'POSITIVE': 0.0005441837129183114}
